In [2]:
pip install selenium

   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ------------------------ --------------- 5.8/9.4 MB 27.1 MB/s eta 0:00:01
   ---------------------------------------- 9.4/9.4 MB 25.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, pandas as pd, os, pickle

# 🔐 사용자 정보 입력
TWITTER_ID = "아이디"
TWITTER_PW = "비번"

COOKIE_PATH = "twitter_cookies.pkl"

# 🔧 Chrome 설정
options = Options()
#options.add_argument("--headless")  # 창 없이 실행
# options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920x1080")
options.add_argument("--incognito")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://twitter.com")
time.sleep(40)
user_ids = [17140361] # 예시

results = []

for user_id in user_ids:
    try:
        driver.get(f"https://twitter.com/i/user/{user_id}")
        time.sleep(3)

        current_url = driver.current_url
        username = current_url.split("/")[-1]

        # 기본 정보 수집
        try:
            display_name = driver.find_element(By.XPATH, '//div[@data-testid="UserName"]//span[1]').text
        except:
            display_name = ""

        try:
            bio = driver.find_element(By.XPATH, '//div[@data-testid="UserDescription"]').text
        except:
            bio = ""

        try:
            profile_pic = driver.find_element(By.XPATH, '//img[@alt="Image"]').get_attribute("src")
        except:
            profile_pic = ""

        try:
            followers = driver.find_element(By.XPATH, '//a[contains(@href,"/followers")]//span[1]/span').text
        except:
            followers = ""

        try:
            following = driver.find_element(By.XPATH, '//a[contains(@href,"/following")]//span[1]/span').text
        except:
            following = ""

        try:
            posts = driver.find_element(By.XPATH, '//a[contains(@href,"/with_replies")]//span[1]/span').text
        except:
            posts = ""

        nums_in_username = sum(c.isdigit() for c in username)
        nums_in_fullname = sum(c.isdigit() for c in display_name)
        external_url = "http" in bio
        is_private = "This account is private" in driver.page_source

        results.append({
            "user_id": user_id,
            "username": username,
            "display_name": display_name,
            "profile_pic": profile_pic,
            "fullname words": len(display_name.split()),
            "nums/len username": f"{nums_in_username/len(username)}",
            "nums/len fullname": f"{nums_in_fullname/len(display_name)}",
            "name==username": display_name.lower() == username.lower(),
            "description length": len(bio),
            "external URL": external_url,
            "private": is_private,
            "#posts": posts,
            "#followers": followers,
            "#follows": following,
        })

        print(f"✅ @{username} 크롤링 완료")

    except Exception as e:
        print(f"❌ {user_id} 실패: {e}")





driver.quit()

df = pd.DataFrame(results)
df.to_csv("x_user_data.csv", index=False, encoding='utf-8-sig')
print("📦 저장 완료: x_user_data.csv")

✅ @CourtneyBaker 크롤링 완료
📦 저장 완료: x_user_data.csv
